Database Connection

In [25]:
import os
import asyncio
import psycopg2
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto
from dotenv import load_dotenv
import sys
# sys.path.append(os.path.abspath("../Script"))
from datetime import datetime
import random
import warnings
warnings.filterwarnings('ignore')

In [26]:
load_dotenv(dotenv_path="../.env")  # adjust the path if needed

True

In [29]:
import psycopg2

# Connect to our PostgreSQL database
host =  os.getenv("pgsql_host")
port =  os.getenv("pgsql_port")
db   =  os.getenv("pgsql_db")
user =  os.getenv("pgsql_user")
password=  os.getenv("pgsql_pass")
conn = psycopg2.connect(
    host=host,
    port=port,
    database=db,
    user=user,
    password=password
)

# Create the cursor BEFORE executing anything
cur = conn.cursor()

# Define our SQL table creation query
create_table_query = '''
CREATE TABLE IF NOT EXISTS messages (
    message_id SERIAL PRIMARY KEY,
    channel_name TEXT,
    message_text TEXT,
    posted_at TIMESTAMP,
    has_image BOOLEAN,
    image_url TEXT
);

CREATE TABLE IF NOT EXISTS images (
    id SERIAL PRIMARY KEY,
    message_id BIGINT UNIQUE,
    channel_name TEXT,
    file_name TEXT,
    file_path TEXT,
    posted_at TIMESTAMP
);
'''
# Execute the query BEFORE closing the cursor
cur.execute(create_table_query)
conn.commit()

print("Table created successfully.")

# Now it's safe to close the cursor and connection
cur.close()
# conn.close()

Table created successfully.


In [24]:
# # Define dummy data
# cur = conn.cursor()
# channels = ['@chemed', '@lobelia4cosmetics', '@tikvahpharma',
# '@ethiopharma', '@medline']
# messages = [
#     "Check out our new antibiotics!",
#     "Skin care promo this weekend!",
#     "We have paracetamol in stock.",
#     "Buy 2 get 1 free: vitamins!",
#     "Fast delivery for all orders."
# ]
# image_urls = [
#     "https://example.com/image1.jpg",
#     "https://example.com/image2.jpg",
#     None,
#     "https://example.com/image4.jpg",
#     None
# ]

# # Insert 5 random records
# for i in range(5):
#     cur.execute(
#         '''
#         INSERT INTO messages (channel_name, message_text, posted_at,
# has_image, image_url)
#         VALUES (%s, %s, %s, %s, %s)
#         ''',
#         (
#             random.choice(channels),
#             messages[i],
#             datetime.now(),
#             image_urls[i] is not None,
#             image_urls[i]
#         )
#     )

# conn.commit()
# print(" 5 records inserted.")
# cur.close()
# conn.close()

In [30]:
import os
import asyncio
import psycopg2
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto
from dotenv import load_dotenv
from datetime import datetime

# Load environment variables
load_dotenv()

# Telegram credentials
# Access the variables
api_id = os.getenv("tg_api_id")
api_hash = os.getenv("tg_api_hash")
session_name = os.getenv("session_name")

# PostgreSQL connection
def get_pgsql_connection():
    try:
        conn = psycopg2.connect(
                host=host,
                port=port,
                database=db,
                user=user,
                password=password
        )
        return conn
    except Exception as e:
        print(f"PostgreSQL connection error: {e}")
        return None

# Async image scraping function
async def scrape_images_async(channel_username, limit=50):
    client = TelegramClient(session_name, api_id, api_hash)
    await client.start()
    print(f"Connected to Telegram. Scraping images from {channel_username}...")

    conn = get_pgsql_connection()
    if conn is None:
        return
    cur = conn.cursor()

    save_dir = os.path.join("images", channel_username.strip("@"))
    os.makedirs(save_dir, exist_ok=True)

    try:
        async for message in client.iter_messages(channel_username,
limit=limit):
            if message.media and isinstance(message.media, MessageMediaPhoto):
                try:
                    file_name = f"{message.id}.jpg"
                    file_path = os.path.join(save_dir, file_name)
                    await client.download_media(message, file_path)

                    cur.execute("""
                        INSERT INTO images (message_id, channel_name,
file_name, file_path, posted_at)
                        VALUES (%s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (
                        message.id,
                        channel_username,
                        file_name,
                        file_path,
                        message.date
                    ))

                    print(f"Saved image: {file_name}")
                except Exception as insert_err:
                    print(f"Error inserting {message.id}: {insert_err}")

        conn.commit()
    except Exception as scrape_err:
        print(f"Scraping failed: {scrape_err}")
    finally:
        cur.close()
        conn.close()
        await client.disconnect()
        print("Disconnected from Telegram.")

In [32]:
await scrape_images_async("@lobelia4cosmetics", limit=30)
await scrape_images_async("@CheMed123", limit=30)
await scrape_images_async("@tikvahpharma", limit=30)

Connected to Telegram. Scraping images from @lobelia4cosmetics...
Saved image: 18535.jpg
Saved image: 18534.jpg
Saved image: 18533.jpg
Saved image: 18532.jpg
Saved image: 18531.jpg
Saved image: 18530.jpg
Saved image: 18529.jpg
Saved image: 18528.jpg
Saved image: 18527.jpg
Saved image: 18526.jpg
Saved image: 18525.jpg
Saved image: 18524.jpg
Saved image: 18523.jpg
Saved image: 18522.jpg
Saved image: 18521.jpg
Saved image: 18520.jpg
Saved image: 18519.jpg
Saved image: 18518.jpg
Saved image: 18517.jpg
Saved image: 18516.jpg
Saved image: 18515.jpg
Saved image: 18514.jpg
Saved image: 18513.jpg
Saved image: 18512.jpg
Saved image: 18511.jpg
Saved image: 18510.jpg
Saved image: 18509.jpg
Saved image: 18508.jpg
Saved image: 18507.jpg
Saved image: 18506.jpg
Disconnected from Telegram.
Connected to Telegram. Scraping images from @CheMed123...
Saved image: 97.jpg
Saved image: 96.jpg
Saved image: 95.jpg
Saved image: 94.jpg
Saved image: 93.jpg
Saved image: 92.jpg
Saved image: 91.jpg
Saved image: 90.jp